#### The original squirrel banking setup assumes the decision to produce offspring yields an immediate payout: you get pregnant, and immediately have an offspring. We now extend to insist that suirrels are pregnant for a day before giving birth; if your food runs below zero while pregnant, both you and your offspring die.

In [1]:
import plotly.express as px
import pandas as pd

In [2]:
p0, p1, p2 = var('p0, p1, p2')

In [3]:
def normalize(P0, P1, P2):
    p0 = P0/(P0 + P1 + P2)
    p1 = P1/(P0 + P1 + P2)
    p2 = 1 - p0 - p1
    return([p0, p1, p2])

def Lnow(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(QQ, 2)
    L[0,0] = p1
    L[1, 0] = p2
    if double_pregnancy == False:
        L[0, 1] = 2*(p1+p2)
        L[1, 1] = 0
    else:
        L[0, 1] = 2*p1 + p2
        L[1, 1] = p2
    return L

def Lpatient(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(QQ, 3)
    L[0,0] = p1
    L[1,0] = p2
    L[2, 0] = 0
    L[0, 1] = p0
    L[1, 1] = p1
    L[2, 1] = p2 
    L[0, 2] = 1 + p0
    if double_pregnancy == False:
        L[1, 2] = p1 + p2
        L[2, 2] = 0
    else:
        L[1, 2] = p1
        L[2, 2] = p2
    return L

def L_betahat(p0, p1, p2, betahat, double_pregnancy = False):
    normalized = normalize(p0, p1, p2)
    p0 = normalized[0]
    p1 = normalized[1]    
    p2 = normalized[2]
    if betahat == 0:
        L = Lnow(p0, p1, p2, double_pregnancy)
    elif betahat == 1:
        L = Lpatient(p0, p1, p2, double_pregnancy)
    else:
        L = zero_matrix(QQ, betahat + 2)
        for i in range(betahat+1):
            L[i, i] = p1
            if i < betahat+1:
                L[i + 1, i] = p2
                if i > 0:
                    L[i-1, i] = p0
        L[0, betahat+1] = 1
        L[betahat - 1, betahat+1] = p0
        if double_pregnancy == False:
            L[betahat, betahat+1] = p1 + p2
            L[betahat+1, betahat+1] = 0
        else:
            L[betahat, betahat + 1] = p1
            L[betahat+1, betahat+1] = p2           
    return(L)

def is_essentially_real(x):
    if x.imag() == 0:
        return(True)
    else:
        return(False)
    
#Use of "if is_essentiallY_real(e)" rather than "if e in RR" is required since some computational errors
#    seem to come up in the eigenvalue computation, giving us things like x + 0.?e-80*I.

def get_leading_eigenvalue(L):
    evals = L.eigenvalues()
    moduli = [e.n() for e in evals if is_essentially_real(e)]
    moduli = [e for e in moduli if e >= 0]
    r = max(moduli)
    return(r)

def patient_or_hasty(p0, p1, p2):
    normalized = normalize(p0, p1, p2)
    print([m.n(digits = 3) for m in normalized])
    p0 = normalized[0]
    p1 = normalized[1]
    p2 = normalized[2]
    Ln = Lnow(p0, p1, p2)
    Lp = Lpatient(p0, p1, p2)
    rn = get_leading_eigenvalue(Ln)
    rp = get_leading_eigenvalue(Lp)
    if rn < rp:
        phrase = "Patience is a virtue."
    elif rn == rp:
        phrase = "It doesn't matter!"
    else:
        phrase = "Strike while the iron is hot."
    numerics = "r_now = {}, r_patient = {}".format(rn.n(digits = 3), rp.n(digits = 3))
    print(numerics)
    return(phrase)

def fitness(p0, p1, p2, betahat, double_pregnancy = False):
    normalized = normalize(p0, p1, p2)
    p0 = normalized[0]
    p1 = normalized[1]
    p2 = normalized[2]
    L = L_betahat(p0, p1, p2, betahat, double_pregnancy)
    r = get_leading_eigenvalue(L)
    return(r)

def first_n_fitness(p0, p1, p2, double_pregnancy = False, n = 5):
    fitnesses = []
    for i in range(n):
        fitnesses.append(fitness(p0, p1, p2, i, double_pregnancy))
    return(fitnesses)
        

In [4]:
def normalize(vec):
    tot = sum(vec)
    vec = vec/tot
    return(vec)

def get_leading_evec(p0, p1, p2, betahat):
    L = L_betahat(p0, p1, p2, betahat)
    r = get_leading_eigenvalue(L)
    evecs = L.eigenvectors_right()
    arrs = [e[0].n() for e in evecs]
    i = arrs.index(r)
    leading_evec = evecs[i][1][0]
    leading_evec = get_stable_age_distn(leading_evec).n()
    return(leading_evec)
def get_optimal_betahat(p0, p1, p2, double_pregancy = False, up_to = 10):
    fnf = first_n_fitness(p0, p1, p2,double_pregancy, up_to)
    betahat = fnf.index(max(fnf))
    return(betahat)
    


In [5]:
first_n_fitness(107.5, 0, 886, n = 2)

[1.26119095748622, 1.26119812516915]

In [15]:
first_n_fitness(1, 2, 1.2, n = 15)

[0.939567612507440,
 1.00830335327835,
 1.01115571579274,
 1.01094889786515,
 1.01014493373333,
 1.00928323152343,
 1.00849415828651,
 1.00780216127526,
 1.00720243242407,
 1.00668300222784,
 1.00623142726464,
 1.00583669095620,
 1.00548955534009,
 1.00518242795107,
 1.00490910615004]

In [14]:
first_n_fitness(230.49,1000,886, n = 2)

[1.13171105365228, 1.13171449050811]

In [137]:
n = 20
triples = []
for i in range(n + 1):
    first = n - i
    for j in range(n - first + 1):
        second = j
        third = i - j
        trip = [first, second, third]
        triples.append(trip)

first = []
second = []
third = []
for i in range(n + 1):
    for j in range(i + 1):
        first.append(n - i)
        second.append(j)
        third.append(i - j)
len(first)

231

In [156]:
L = L_betahat(0, 19, 1, 2)
evecs = L.eigenvectors_right()

In [157]:
get_leading_eigenvalue(L)

1.0223003840704707

In [161]:
float(evecs[1][0]) == get_leading_eigenvalue(L)

False

In [165]:
get_leading_evec(0, 19, 1, 2)

ValueError: 1.02230038407047 is not in list

In [118]:
first[k], second[k], third[k]

(0, 19, 1)

In [116]:
betahat = 2
for k in range(len(first)):
    n = get_leading_evec(first[k], second[k], third[k], betahat)
    print(n[0], n[-2])

(1.00000000000000, 0.000000000000000)
(0.717846293925489, 0.0377340497613420)
(1.00000000000000, 0.000000000000000)
(0.618369403568888, 0.0683462809662520)
(0.711008396922708, 0.0402761609358561)
(1.00000000000000, 0.000000000000000)
(0.548091425880116, 0.0955114988868822)
(0.609589405596708, 0.0727784947674140)
(0.703703188708457, 0.0429605939776824)
(1.00000000000000, 0.000000000000000)
(0.492843462747318, 0.120302372616402)
(0.538222354457530, 0.101503576007617)
(0.600223941444094, 0.0775336390205008)
(0.695853136567601, 0.0458460840077637)
(1.00000000000000, 0.000000000000000)
(0.447266372422824, 0.143245715353895)
(0.482359711338408, 0.127623936974028)
(0.527729980928786, 0.107971970304973)
(0.590191021605662, 0.0826918281526240)
(0.687369959075294, 0.0489943043362569)
(1.00000000000000, 0.000000000000000)
(0.408682347540972, 0.164643037605478)
(0.436502853803649, 0.151711954034877)
(0.471267762971968, 0.135546553522320)
(0.516539774035427, 0.115014037347785)
(0.579395944748365, 0

ValueError: 1.0223003840704707 is not in list

In [92]:
n = get_leading_evec(1, 3, 2, 0)
n

(0.756601886794340, 0.243398113205660)

In [84]:
n = get_leading_evec(1.78, 2, 2, 2)
n

(0.299674859113913, 0.331160977626351, 0.274435180552712, 0.0947289827070235)

In [85]:
n[-2]*normalize(1.78, 2, 2)[2]

0.0949602700874435

### There is what looks to be a serious problem. The computation of first_five_fitness(31, 38, 31) gives us the list:
[0.871102048154313,
 0.9763953680972295,
 0.9877454676425148,
 0.9925330163760127,
 0.9949830556127044,
 0.9964003778575083,
 0.8314419583831054,
 0.9978903560054913,

### and it is unclear why the second last entry is so abarrent. Look into it!

#### I have found the answer to the above riddle. What I figure really ought to be the leading eigenvalue is listed as "0.997292687532750? + 0.?e-49*I"  which I am going to assume is a computational error. I'll have to define a function along the lines of "really_ought_to_be_real" to sort out this issue.

In [21]:
get_optimal_betahat(4, 0, 6)

4

In [28]:
55/45.n()

1.22222222222222

In [22]:
enns = [get_leading_evec(45, 0, 55, i) for i in range(10)]

In [25]:
delta0 = [enns[k][0] - enns[k+1][0] for k in range(9)]
delta_betahat = [enns[k][-2] - enns[k+1][-2] for k in range(9)]

In [29]:
ratio = [delta0[k]/delta_betahat[k] for k in range(len(delta0))]

In [31]:
[1/k for k in ratio]

[1.66134974597242,
 0.621704015280882,
 0.722060042090105,
 0.800077652875833,
 0.862027718943695,
 0.911948894528958,
 0.952544702482119,
 0.985708666018279,
 1.01282544353536]

In [26]:
delta0

[0.138366719437758,
 0.165479877754828,
 0.0749494431957915,
 0.0416744284081150,
 0.0262266687551141,
 0.0179339292916103,
 0.0130167459824685,
 0.00988209436361911,
 0.00777046622457300]

In [27]:
delta_betahat

[0.229875514188955,
 0.102879504448366,
 0.0541179981085831,
 0.0333427788657066,
 0.0226081154424629,
 0.0163548269920445,
 0.0123990324291557,
 0.00974086605262975,
 0.00787012590037969]

In [20]:
for i in range(10):
    n = get_leading_evec(40, 0, 60, i)
    print(n[0],n[-2])

(0.585786437626905, 0.585786437626905)
(0.423733018758950, 0.365589705360278)
(0.266100886876877, 0.267388154854233)
(0.199244094936239, 0.215356291489826)
(0.163615981762535, 0.183014379302378)
(0.141678937363756, 0.160825630340308)
(0.126742786912180, 0.144539634397173)
(0.115797810081109, 0.131986078273828)
(0.107324417405556, 0.121945791056943)
(0.100486258948454, 0.113683227647284)


#### Just for checking to see if I have my matrix form correct:

In [68]:
def sym_Lnow(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(SR, 2)
    L[0,0] = p1
    L[1, 0] = p2
    if double_pregnancy == False:
        L[0, 1] = 2*(p1+p2)
        L[1, 1] = 0
    else:
        L[0, 1] = 2*p1 + p2
        L[1, 1] = p2
    return L

def sym_Lpatient(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(SR, 3)
    L[0,0] = p1
    L[1,0] = p2
    L[2, 0] = 0
    L[0, 1] = p0
    L[1, 1] = p1
    L[2, 1] = p2 
    L[0, 2] = 1 + p0
    if double_pregnancy == False:
        L[1, 2] = p1 + p2
        L[2, 2] = 0
    else:
        L[1, 2] = p1
        L[2, 2] = p2
    return L

def sym_L_betahat(p0, p1, p2, betahat, double_pregnancy = False):
    if betahat == 0:
        L = sym_Lnow(p0, p1, p2, double_pregnancy)
    elif betahat == 1:
        L = sym_Lpatient(p0, p1, p2, double_pregnancy)
    else:
        L = zero_matrix(SR, betahat + 2)
        for i in range(betahat+1):
            L[i, i] = p1
            if i < betahat+1:
                L[i + 1, i] = p2
                if i > 0:
                    L[i-1, i] = p0
        L[0, betahat+1] = 1
        L[betahat - 1, betahat+1] = p0
        if double_pregnancy == False:
            L[betahat, betahat+1] = p1 + p2
            L[betahat+1, betahat+1] = 0
        else:
            L[betahat, betahat + 1] = p1
            L[betahat+1, betahat+1] = p2           
    return(L)

In [70]:
sym_L_betahat(p0, p1, p2, 2)

[     p1      p0       0       1]
[     p2      p1      p0      p0]
[      0      p2      p1 p1 + p2]
[      0       0      p2       0]